# Estimating Tour Mode Choice

Integration with [larch](https://larch.newman.me) for model estimation. See [estimation tools review](https://github.com/ActivitySim/activitysim/wiki/Estimation-Tools-Review) for more information about larch.

# Run the Example

Output an estimation data bundle (EBD), which contains:
  - model settings - tour_mode_choice_model_settings.yaml
  - coefficients - tour_mode_choice_coefficients.csv
  - coefficients template by tour purpose - tour_mode_choice_coefficients_template.csv
  - utilities specification - tour_mode_choice_SPEC.csv
  - chooser data - tour_mode_choice_values_combined.csv

# Read EDB 

In [1]:
import larch  # !conda install larch #for estimation
import pandas as pd
import numpy as np
import yaml 
import larch.util.excel
import larch_asim  # utility functions in a local module
import os

from larch import P,X

In [2]:
edb_directory = "estimation_data_bundle/tour_mode_choice/"

def read_csv(filename, **kwargs):
    return pd.read_csv(os.path.join(edb_directory, filename), **kwargs)

In [3]:
coefficients = read_csv(
    "tour_mode_choice_coefficients.csv",
    index_col='coefficient_name',
)
coef_template = read_csv(
    "tour_mode_choice_coefficients_template.csv", 
    index_col='coefficient_name',
)
spec = read_csv("tour_mode_choice_SPEC.csv")
values = read_csv("tour_mode_choice_values_combined.csv")

c:\programdata\anaconda3\envs\larchtest\lib\site-packages\IPython\core\interactiveshell.py:3254: DtypeWarning: Columns (326) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


## settings

In [4]:
settings = yaml.load( 
    open(os.path.join(edb_directory, "tour_mode_choice_model_settings.yaml"),"r"), 
    Loader=yaml.SafeLoader,
)

settings

{'LOGIT_TYPE': 'NL',
 'NESTS': {'name': 'root',
  'coefficient': 'coef_nest_root',
  'alternatives': [{'name': 'AUTO',
    'coefficient': 'coef_nest_AUTO',
    'alternatives': [{'name': 'DRIVEALONE',
      'coefficient': 'coef_nest_AUTO_DRIVEALONE',
      'alternatives': ['DRIVEALONEFREE', 'DRIVEALONEPAY']},
     {'name': 'SHAREDRIDE2',
      'coefficient': 'coef_nest_AUTO_SHAREDRIDE2',
      'alternatives': ['SHARED2FREE', 'SHARED2PAY']},
     {'name': 'SHAREDRIDE3',
      'coefficient': 'coef_nest_AUTO_SHAREDRIDE3',
      'alternatives': ['SHARED3FREE', 'SHARED3PAY']}]},
   {'name': 'NONMOTORIZED',
    'coefficient': 'coef_nest_NONMOTORIZED',
    'alternatives': ['WALK', 'BIKE']},
   {'name': 'TRANSIT',
    'coefficient': 'coef_nest_TRANSIT',
    'alternatives': [{'name': 'WALKACCESS',
      'coefficient': 'coef_nest_TRANSIT_WALKACCESS',
      'alternatives': ['WALK_LOC',
       'WALK_LRF',
       'WALK_EXP',
       'WALK_HVY',
       'WALK_COM']},
     {'name': 'DRIVEACCESS',
      

## coefficients

In [5]:
coefficients

,value,constrain
coefficient_name,,
coef_nest_root,1.000,T
coef_nest_AUTO,0.720,T
coef_nest_AUTO_DRIVEALONE,0.350,T
coef_nest_AUTO_SHAREDRIDE2,0.350,T
coef_nest_AUTO_SHAREDRIDE3,0.350,T
...,...,...
walk_transit_CBD_ASC_atwork,0.564,F
drive_transit_CBD_ASC_eatout_escort_othdiscr_othmaint_shopping_social,0.525,F
drive_transit_CBD_ASC_school_univ,0.672,F


## coef_template

In [6]:
coef_template

,eatout,escort,othdiscr,othmaint,school,shopping,social,univ,work,atwork
coefficient_name,,,,,,,,,,
coef_nest_root,coef_nest_root,coef_nest_root,coef_nest_root,coef_nest_root,coef_nest_root,coef_nest_root,coef_nest_root,coef_nest_root,coef_nest_root,coef_nest_root
coef_nest_AUTO,coef_nest_AUTO,coef_nest_AUTO,coef_nest_AUTO,coef_nest_AUTO,coef_nest_AUTO,coef_nest_AUTO,coef_nest_AUTO,coef_nest_AUTO,coef_nest_AUTO,coef_nest_AUTO
coef_nest_AUTO_DRIVEALONE,coef_nest_AUTO_DRIVEALONE,coef_nest_AUTO_DRIVEALONE,coef_nest_AUTO_DRIVEALONE,coef_nest_AUTO_DRIVEALONE,coef_nest_AUTO_DRIVEALONE,coef_nest_AUTO_DRIVEALONE,coef_nest_AUTO_DRIVEALONE,coef_nest_AUTO_DRIVEALONE,coef_nest_AUTO_DRIVEALONE,coef_nest_AUTO_DRIVEALONE
coef_nest_AUTO_SHAREDRIDE2,coef_nest_AUTO_SHAREDRIDE2,coef_nest_AUTO_SHAREDRIDE2,coef_nest_AUTO_SHAREDRIDE2,coef_nest_AUTO_SHAREDRIDE2,coef_nest_AUTO_SHAREDRIDE2,coef_nest_AUTO_SHAREDRIDE2,coef_nest_AUTO_SHAREDRIDE2,coef_nest_AUTO_SHAREDRIDE2,coef_nest_AUTO_SHAREDRIDE2,coef_nest_AUTO_SHAREDRIDE2
coef_nest_AUTO_SHAREDRIDE3,coef_nest_AUTO_SHAREDRIDE3,coef_nest_AUTO_SHAREDRIDE3,coef_nest_AUTO_SHAREDRIDE3,coef_nest_AUTO_SHAREDRIDE3,coef_nest_AUTO_SHAREDRIDE3,coef_nest_AUTO_SHAREDRIDE3,coef_nest_AUTO_SHAREDRIDE3,coef_nest_AUTO_SHAREDRIDE3,coef_nest_AUTO_SHAREDRIDE3,coef_nest_AUTO_SHAREDRIDE3
...,...,...,...,...,...,...,...,...,...,...
express_bus_ASC,express_bus_ASC_eatout_escort_othdiscr_othmain...,express_bus_ASC_eatout_escort_othdiscr_othmain...,express_bus_ASC_eatout_escort_othdiscr_othmain...,express_bus_ASC_eatout_escort_othdiscr_othmain...,express_bus_ASC_school_univ,express_bus_ASC_eatout_escort_othdiscr_othmain...,express_bus_ASC_eatout_escort_othdiscr_othmain...,express_bus_ASC_school_univ,express_bus_ASC_work,express_bus_ASC_eatout_escort_othdiscr_othmain...
heavy_rail_ASC,heavy_rail_ASC_eatout_escort_othdiscr_othmaint...,heavy_rail_ASC_eatout_escort_othdiscr_othmaint...,heavy_rail_ASC_eatout_escort_othdiscr_othmaint...,heavy_rail_ASC_eatout_escort_othdiscr_othmaint...,heavy_rail_ASC_school_univ,heavy_rail_ASC_eatout_escort_othdiscr_othmaint...,heavy_rail_ASC_eatout_escort_othdiscr_othmaint...,heavy_rail_ASC_school_univ,heavy_rail_ASC_work,heavy_rail_ASC_eatout_escort_othdiscr_othmaint...
commuter_rail_ASC,commuter_rail_ASC_eatout_escort_othdiscr_othma...,commuter_rail_ASC_eatout_escort_othdiscr_othma...,commuter_rail_ASC_eatout_escort_othdiscr_othma...,commuter_rail_ASC_eatout_escort_othdiscr_othma...,commuter_rail_ASC_school_univ,commuter_rail_ASC_eatout_escort_othdiscr_othma...,commuter_rail_ASC_eatout_escort_othdiscr_othma...,commuter_rail_ASC_school_univ,commuter_rail_ASC_work,commuter_rail_ASC_eatout_escort_othdiscr_othma...


## spec

In [7]:
# Remove apostrophes from Label names
spec['Label'] = spec['Label'].str.replace("'","")

In [8]:
spec

,Label,Description,Expression,DRIVEALONEFREE,DRIVEALONEPAY,SHARED2FREE,SHARED2PAY,SHARED3FREE,SHARED3PAY,WALK,...,WALK_HVY,WALK_COM,DRIVE_LOC,DRIVE_LRF,DRIVE_EXP,DRIVE_HVY,DRIVE_COM,TAXI,TNC_SINGLE,TNC_SHARED
0,#,Drive alone no toll,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,util_DRIVEALONEFREE_Unavailable,DRIVEALONEFREE - Unavailable,sov_available == False,-999,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,util_DRIVEALONEFREE_Unavailable_for_zero_auto_...,DRIVEALONEFREE - Unavailable for zero auto hou...,auto_ownership == 0,-999,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,util_DRIVEALONEFREE_Unavailable_for_persons_le...,DRIVEALONEFREE - Unavailable for persons less ...,age < 16,-999,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,util_DRIVEALONEFREE_Unavailable_for_joint_tours,DRIVEALONEFREE - Unavailable for joint tours,is_joint == True,-999,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
334,#,FIXME - skims aren't symmetrical,so we have to make sure they can get back,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
335,util_Walk_not_available_for_long_distances,Walk not available for long distances,@od_skims.max('DISTWALK') > 3,NaN,NaN,NaN,NaN,NaN,NaN,-999,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
336,util_Bike_not_available_for_long_distances,Bike not available for long distances,@od_skims.max('DISTBIKE') > 8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
337,util_Drive_alone_not_available_for_escort_tours,Drive alone not available for escort tours,is_escort,-999,-999,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
# Check for double-parameters
ss = spec.query("Label!='#'").iloc[:,3:].stack().str.split("*")
st = ss.apply(lambda x: len(x))>1
assert len(ss[st]) == 0

## values

In [10]:
# Remove apostrophes from column names
values.columns = values.columns.str.replace("'","")
values.fillna(0, inplace=True)
values

,tour_id,model_choice,util_DRIVEALONEFREE_Unavailable,util_DRIVEALONEFREE_Unavailable_for_zero_auto_households,util_DRIVEALONEFREE_Unavailable_for_persons_less_than_16,util_DRIVEALONEFREE_Unavailable_for_joint_tours,util_DRIVEALONEFREE_Unavailable_if_didnt_drive_to_work,util_DRIVEALONEFREE_In_vehicle_time,util_DRIVEALONEFREE_Terminal_time,util_DRIVEALONEFREE_Operating_cost,...,walk_heavyrail_available,walk_lrf_available,walk_ferry_available,drive_local_available,drive_commuter_available,drive_express_available,drive_heavyrail_available,drive_lrf_available,drive_ferry_available,destination_in_cbd
0,1378,WALK,0.0,1.0,0.0,0.0,0.0,5.350000,14.36728,10.961482,...,False,False,False,False,False,False,False,False,False,0
1,10133,SHARED3FREE,0.0,0.0,0.0,0.0,0.0,7.330000,13.16400,0.671488,...,False,False,False,True,True,False,True,False,False,1
2,10136,SHARED2FREE,0.0,0.0,0.0,0.0,0.0,16.719999,16.68040,1.825009,...,False,False,False,True,True,False,True,True,False,1
3,26637,DRIVEALONEFREE,0.0,0.0,0.0,0.0,0.0,16.110001,15.88032,5.886337,...,True,False,False,True,True,False,True,False,False,0
4,26642,WALK,0.0,0.0,0.0,0.0,0.0,2.180000,8.69976,0.707018,...,False,False,False,False,False,False,False,False,False,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5299,309177212,WALK,0.0,1.0,0.0,0.0,0.0,3.830000,18.32624,1.438632,...,False,True,False,False,False,False,False,False,False,1
5300,309177215,WALK,0.0,1.0,0.0,0.0,0.0,7.070000,21.84808,2.360572,...,False,False,False,False,False,False,False,False,False,1
5301,309790009,BIKE,0.0,0.0,0.0,0.0,0.0,15.309999,21.41740,63.429717,...,False,False,False,True,True,False,True,False,False,1
5302,309800789,SHARED3FREE,0.0,0.0,0.0,0.0,0.0,18.000000,16.96564,69.387058,...,False,True,False,True,True,False,True,True,False,1


# Data Setup

## Alternatives

In [11]:
alt_names = list(spec.columns[3:])
alt_codes = np.arange(1,len(alt_names)+1)
alt_names_to_codes = dict(zip(alt_names, alt_codes))
alt_codes_to_names = dict(zip(alt_codes, alt_names))
alt_names_to_codes

{'DRIVEALONEFREE': 1,
 'DRIVEALONEPAY': 2,
 'SHARED2FREE': 3,
 'SHARED2PAY': 4,
 'SHARED3FREE': 5,
 'SHARED3PAY': 6,
 'WALK': 7,
 'BIKE': 8,
 'WALK_LOC': 9,
 'WALK_LRF': 10,
 'WALK_EXP': 11,
 'WALK_HVY': 12,
 'WALK_COM': 13,
 'DRIVE_LOC': 14,
 'DRIVE_LRF': 15,
 'DRIVE_EXP': 16,
 'DRIVE_HVY': 17,
 'DRIVE_COM': 18,
 'TAXI': 19,
 'TNC_SINGLE': 20,
 'TNC_SHARED': 21}

## Nesting Tree

In [12]:
tree = larch_asim.construct_nesting_tree(alt_names, settings['NESTS'])

tree

In [13]:
tree.elemental_names()

{1: 'DRIVEALONEFREE',
 2: 'DRIVEALONEPAY',
 3: 'SHARED2FREE',
 4: 'SHARED2PAY',
 5: 'SHARED3FREE',
 6: 'SHARED3PAY',
 7: 'WALK',
 8: 'BIKE',
 9: 'WALK_LOC',
 10: 'WALK_LRF',
 11: 'WALK_EXP',
 12: 'WALK_HVY',
 13: 'WALK_COM',
 14: 'DRIVE_LOC',
 15: 'DRIVE_LRF',
 16: 'DRIVE_EXP',
 17: 'DRIVE_HVY',
 18: 'DRIVE_COM',
 19: 'TAXI',
 20: 'TNC_SINGLE',
 21: 'TNC_SHARED'}

## Purposes

In [14]:
purposes = list(coef_template.columns)
purposes

['eatout',
 'escort',
 'othdiscr',
 'othmaint',
 'school',
 'shopping',
 'social',
 'univ',
 'work',
 'atwork']

## Purpose-specific Models

In [15]:
m = {purpose:larch.Model(graph=tree) for purpose in purposes}

In [16]:
for alt_code, alt_name in tree.elemental_names().items():
    # Read in base utility function for this alt_name
    u = larch_asim.linear_utility_from_spec(
        spec, x_col='Label', p_col=alt_name, 
        ignore_x=('#',), 
    )
    for purpose in purposes:
        # Modify utility function based on template for purpose
        u_purp = sum(
            (
                P(coef_template[purpose].get(i.param,i.param)) 
                * i.data * i.scale
            )
            for i in u
        )
        m[purpose].utility_co[alt_code] = u_purp


## Set Parameter Values

In [17]:
for model in m.values():
    larch_asim.explicit_value_parameters(model)

In [18]:
larch_asim.apply_coefficients(coefficients, m)

## DataFrames

In [19]:
values['model_choice_code'] = values.model_choice.map(alt_names_to_codes)

In [20]:
d = larch.DataFrames(
    co=values.set_index('tour_id'),
    av=True,
    alt_codes=alt_codes,
    alt_names=alt_names,
)

In [21]:
for purpose, model in m.items():
    model.dataservice = d.selector_co(f"tour_type=='{purpose}'")
    model.choice_co_code = 'model_choice_code'

In [22]:
from larch.model.model_group import ModelGroup
mg = ModelGroup(m.values())

# Estimate

Note: The demo test data here is 100 households and the model has 
57 estimated parameters -- the result is a very over-specified
model which does not have a numerically stable likelihood maximizing
solution.

In [23]:
mg.estimate()

req_data does not request avail_ca or avail_co but it is set and being provided
req_data does not request avail_ca or avail_co but it is set and being provided
req_data does not request avail_ca or avail_co but it is set and being provided
req_data does not request avail_ca or avail_co but it is set and being provided
req_data does not request avail_ca or avail_co but it is set and being provided
req_data does not request avail_ca or avail_co but it is set and being provided
req_data does not request avail_ca or avail_co but it is set and being provided
req_data does not request avail_ca or avail_co but it is set and being provided
req_data does not request avail_ca or avail_co but it is set and being provided
req_data does not request avail_ca or avail_co but it is set and being provided


,value,initvalue,nullvalue,minimum,maximum,holdfast,note,best
-999,-999.000000,-999.0,-999.0,-999.0,-999.0,1,,-999.000000
1,1.000000,1.0,1.0,1.0,1.0,1,,1.000000
bike_ASC_auto_deficient_eatout,-17.907787,0.0,0.0,NaN,NaN,0,,-17.907787
bike_ASC_auto_sufficient_eatout,-1.721446,0.0,0.0,NaN,NaN,0,,-1.721446
bike_ASC_no_auto_eatout,19.418071,0.0,0.0,NaN,NaN,0,,19.418071
...,...,...,...,...,...,...,...,...
walk_ASC_no_auto_atwork,6.669213,0.0,0.0,NaN,NaN,0,,6.669213
walk_transit_ASC_auto_deficient_atwork,-2.998829,0.0,0.0,NaN,NaN,0,,-2.998829
walk_transit_ASC_auto_sufficient_atwork,-3.401027,0.0,0.0,NaN,NaN,0,,-3.401027
walk_transit_ASC_no_auto_atwork,2.704188,0.0,0.0,NaN,NaN,0,,2.704188


c:\programdata\anaconda3\envs\larchtest\lib\site-packages\numpy\core\_methods.py:38: RuntimeWarning: invalid value encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims, initial, where)
c:\programdata\anaconda3\envs\larchtest\lib\site-packages\ipykernel_launcher.py:1: PossibleOverspecification: WARNING: Model is possibly over-specified (hessian is nearly singular).
  """Entry point for launching an IPython kernel.
c:\programdata\anaconda3\envs\larchtest\lib\site-packages\larch\linalg\__init__.py:18: UserWarning: minimum eig 0.0 in general_inverse
  warnings.warn(f"minimum eig {min_eig} in general_inverse")
c:\programdata\anaconda3\envs\larchtest\lib\site-packages\ipykernel_launcher.py:1: RuntimeWarning: invalid value encountered in sqrt
  """Entry point for launching an IPython kernel.


┣                x: -999                                      -999.000000
┃                   1                                            1.000000
┃                   bike_ASC_auto_deficient_eatout             -17.907787
┃                   bike_ASC_auto_sufficient_eatout             -1.721446
┃                   bike_ASC_no_auto_eatout                     19.418071
┃                                                                 ...    
┃                   walk_ASC_no_auto_atwork                      6.669213
┃                   walk_transit_ASC_auto_deficient_atwork      -2.998829
┃                   walk_transit_ASC_auto_sufficient_atwork     -3.401027
┃                   walk_transit_ASC_no_auto_atwork              2.704188
┃                   walk_transit_CBD_ASC_atwork                  0.564000
┃                   Length: 293, dtype: float64
┣          loglike: -5688.126512356238
┣        d_loglike: -999                                       0.000000e+00
┃                   1                                          0.000000e+00
┃                   bike_ASC_auto_deficient_eatout            -2.343835e-08
┃                   bike_ASC_auto_sufficient_eatout           -9.677219e-04
┃                   bike_ASC_no_auto_eatout                    4.688340e-03
┃                                                                  ...     
┃                   walk_ASC_no_auto_atwork                    0.000000e+00
┃                   walk_transit_ASC_auto_deficient_atwork     0.000000e+00
┃                   walk_transit_ASC_auto_sufficient_atwork    0.000000e+00
┃                   walk_transit_ASC_no_auto_atwork            0.000000e+00
┃                   walk_transit_CBD_ASC_atwork                0.000000e+00
┃                   Length: 293, dtype: float64
┣              nit: 101
┣             nfev: 252
┣             njev: 101
┣           status: 9
┣          message: 'Iteration limit exceeded'
┣          success: False
┣     elapsed_time: datetime.timedelta(seconds=84, microseconds=925535)
┣           method: 'slsqp'
┣          n_cases: 5304
┣ iteration_number: 101
┣          logloss: 1.0724220422994415

# Outputs

In [24]:
# The test model is wildly overspecified.
#
# mg.possible_overspecification 

In [25]:
est_names = [j for j in coefficients.index if j in mg.pf.index]

In [26]:
# Write re-estimated value back into the coefficients file.
coefficients.loc[est_names, 'value'] = mg.pf.loc[est_names, 'value']

In [27]:
# Write out replacement coefficients file and model summaries
os.makedirs(os.path.join(edb_directory,'estimated'), exist_ok=True)

coefficients.reset_index().to_csv(
    os.path.join(
        edb_directory, 
        'estimated',
        "tour_mode_choice_coefficients_revised.csv",
    ),
    index=False,
)

for purpose, model in m.items():
    model.to_xlsx(
        os.path.join(
            edb_directory, 
            'estimated',
            f"tour_mode_choice_{purpose}_model_estimation.xlsx",
        )
    )